# Aplicação da ResNet 50

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Diretórios de treino e teste
train_dir = 'PokemonSplit/train'
test_dir = 'PokemonSplit/test'

In [3]:
# Hiperparâmetros
img_height, img_width = 224, 224
batch_size = 32
epochs = 10

# Criação dos geradores de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Carregamento do modelo ResNet-50 pré-treinado no ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Adição de camadas customizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(150, activation='softmax')(x)

# Criação do modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Congelamento das camadas base do ResNet-50
for layer in base_model.layers:
    layer.trainable = False

# Compilação do modelo
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento do modelo
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator)


Found 5391 images belonging to 150 classes.
Found 814 images belonging to 150 classes.
Epoch 1/10


c:\Users\madus\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


169/169 ━━━━━━━━━━━━━━━━━━━━ 415s 2s/step - accuracy: 0.0079 - loss: 5.1506 - val_accuracy: 0.0221 - val_loss: 4.9445
Epoch 2/10
169/169 ━━━━━━━━━━━━━━━━━━━━ 371s 2s/step - accuracy: 0.0244 - loss: 4.9393 - val_accuracy: 0.0332 - val_loss: 4.8458
Epoch 3/10
169/169 ━━━━━━━━━━━━━━━━━━━━ 359s 2s/step - accuracy: 0.0273 - loss: 4.8415 - val_accuracy: 0.0405 - val_loss: 4.7783
Epoch 4/10
169/169 ━━━━━━━━━━━━━━━━━━━━ 353s 2s/step - accuracy: 0.0287 - loss: 4.7616 - val_accuracy: 0.0418 - val_loss: 4.7338
Epoch 5/10
169/169 ━━━━━━━━━━━━━━━━━━━━ 350s 2s/step - accuracy: 0.0419 - loss: 4.6750 - val_accuracy: 0.0369 - val_loss: 4.6691
Epoch 6/10
169/169 ━━━━━━━━━━━━━━━━━━━━ 341s 2s/step - accuracy: 0.0486 - loss: 4.6306 - val_accuracy: 0.0577 - val_loss: 4.6419
Epoch 7/10
169/169 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - accuracy: 0.0580 - loss: 4.5243 - val_accuracy: 0.0627 - val_loss: 4.5835
Epoch 8/10
169/169 ━━━━━━━━━━━━━━━━━━━━ 372s 2s/step - accuracy: 0.0650 - loss: 4.4802 - val_accuracy: 0.060

In [4]:
# Avaliação do modelo
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')

26/26 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.0767 - loss: 4.4877
Test accuracy: 7.13%


In [ ]:
# Após treinar o modelo
model.save('pokemon_classifier.h5')

Teste com 1 imagem

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
from tensorflow.keras.models import load_model

# Carregar o modelo salvo
model = load_model('pokemon_classifier.h5')

In [2]:
# Caminho da imagem de exemplo (pikachu)
img_path = 'PokemonSplit/test/Pikachu/00000059.jpg'

In [3]:
# Função para carregar e preparar uma imagem para a previsão
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalização
    return img_array

# Carregar e preprocessar a imagem
img_array = load_and_preprocess_image(img_path)

# Fazer a previsão com o modelo treinado
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Mapeamento de índices de classes para nomes de classes
class_indices = train_generator.class_indices
class_names = {v: k for k, v in class_indices.items()}

# Mostrar a imagem e a previsão
plt.imshow(image.load_img(img_path))
plt.title(f'Predicted: {class_names[predicted_class[0]]}')
plt.axis('off')
plt.show()

# Avaliação do modelo no conjunto de teste
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')


NameError: name 'model' is not defined